In [14]:
import asyncio, edge_tts

In [15]:
def choose_voice(age_years: float, gender: str, locale='en-US'):
    """
    Returns a Microsoft Edge TTS voice name keyed by locale, age band, and gender, all in English.
    """
    # coarse bands: teen, young adult, adult, senior
    if age_years is None or gender not in ('male','female'):
        # safe fallback
        return f"{locale}-AriaNeural"  # neutral/friendly female

    if age_years < 10:
        return f"{locale}-AnaNeural" if gender=='female' else f"{locale}-FableTurboMultilingualNeural"
    elif age_years < 18:
        return f"{locale}-JennyMultilingualNeural" if gender=='female' else f"{locale}-RyanMultilingualNeural"
    elif age_years < 30:
        return f"{locale}-AriaNeural" if gender=='female' else f"{locale}-GuyNeural"
    elif age_years < 55:
        return f"{locale}-SaraNeural" if gender=='female' else f"{locale}-DavisNeural"

In [16]:
async def tts_to_file(text: str, voice: str, out_wav: str, rate="+0%", pitch="+0Hz"):
    if voice == 'en-US-FableTurboMultilingualNeural':
        pitch = "+35Hz"
    communicate = edge_tts.Communicate(text, voice=voice, rate=rate, pitch=pitch)
    print(voice)
    with open(out_wav, "wb") as f:
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                f.write(chunk["data"])

def speak_text(text, voice, out_wav="output.wav"):
    coro = tts_to_file(text, voice, out_wav)
    try:
        loop = asyncio.get_running_loop()  # notebook: loop is running
    except RuntimeError:
        return asyncio.run(coro)           # script: no loop -> run now
    else:
        # notebook: schedule and return the Task so the caller can await it
        return asyncio.create_task(coro)

In [ ]:
out_wav = "/home/ssever/SilentSpeak/audio.wav"
audio = speak_text("Hi, my name is Chris!", choose_voice(25, 'male'), out_wav)
await audio

In [ ]:
from IPython.display import Audio, display
display(Audio("/home/ssever/SilentSpeak/audio.wav", autoplay=False))